In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import os
import warnings
import harmonypy as hm
import scanorama
import scvi
from time import time



warnings.filterwarnings('ignore')

sc.set_figure_params(figsize=(8, 8), dpi=80, dpi_save=600)

/home/ojanen/data/new_conda/envs/heartmap/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/ojanen/data/new_conda/envs/heartmap/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/ojanen/data/new_conda/envs/heartmap/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create small set of testing data

In [2]:
files = os.listdir('../scCAD/jupyter/qclus/benchmarking/qclus_results/dataset_adatas/selected_samples')

adata = [sc.read_h5ad('../scCAD/jupyter/qclus/benchmarking/qclus_results/dataset_adatas/selected_samples/' + file) for file in files]

#concatenate all the samples
adata = adata[0].concatenate(adata[1:], join='outer')

In [3]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_MT'], n_jobs=48)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)

In [4]:
datasets = ['Lit_dataset', 'Tu_dataset', 'Chaff_dataset', 'Koenig_dataset', 'Ku_dataset', 'Hill_dataset']
dataset_names_corrected = ['Litviňuková et al.', 'CAREBANK', 'Chaffin et al.', 'Koenig et al.', 'PERIHEART', 'Hill et al.']

# Create a mapping dictionary
mapping_dict = dict(zip(datasets, dataset_names_corrected))

# Replace values in adata.obs['dataset']
adata.obs['dataset_id'] = adata.obs['dataset_id'].map(mapping_dict)

In [5]:
adata.write('data/selected_samples.h5ad')
#adata = sc.read_h5ad('data/selected_samples.h5ad')